In [10]:
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import warnings
import os

In [11]:
colorscale = ['#77DD77','#33AF13','#F6D20E','#F17700','#FE6B64','#F12424']
PAPER_BGCOLOR = '#f5f2d0'
BGCOLOR = 'LightSteelBlue'

In [12]:
fig = go.Figure(data=[go.Table(
    columnorder = [1,2,3,4],
    columnwidth = [50,70,60,400],
    
    header=dict(values=['<b>AQI</b>', '<b>Remark</b>','<b>Colour Code</b>','<b>Possible Health Effects</b>'],
                line_color='darkslategray',
                fill_color='skyblue',
                align='left'),
    cells=dict(values=[['0-50','51-100','101-200','201-300','301-400','401-500'],
                       ['Good','Satisfactory','Moderate','Poor','Very Poor','Severe'],
                       ['','','','','',''],
                       ['Minimal impact','Minor breathing discomfort to sensitive people',\
                       'Breathing discomfort to the people with lungs, asthma and heart diseases',\
                       'Breathing discomfort to most people on prolonged exposure',\
                       'Respiratory illness on prolonged exposure','Affects healthy people and seriously impacts those with existing diseases']],
               line_color='darkslategray',
               fill_color=['rgb(255,255,255)',
                           'rgb(255,255,255)',
                            [color for color in colorscale],
                           'rgb(255,255,255)'],
               align='left'))
])

fig.update_layout(height=180,paper_bgcolor='LightSteelBlue',margin=dict(l=5,r=5,t=5,b=5))
fig.show()

In [13]:
os.chdir(r'C:\Users\shiva\Desktop\AQI mp')
df_cd = pd.read_csv('city_day.csv')
df_ch = pd.read_csv('city_hour.csv')
df_sd = pd.read_csv('station_day.csv')
df_sh = pd.read_csv('station_hour.csv')
df_st = pd.read_csv('stations.csv')

In [14]:
group = df_st.groupby(['State','Status'],as_index=False)['StationId'].count()
group.sort_values(['StationId'],inplace=True,ascending=False)
fig = go.Figure()
active = group[group['Status']=='Active']
fig.add_trace(go.Bar(x=active.State,y=active.StationId,name='Active',
                     marker_color='rgb(119, 221, 119)',
                     marker_line_color='black',
                     marker_line_width=1.5, 
                     opacity=0.9
                    ))
na = group[group['Status']=='NA']
fig.add_trace(go.Bar(x=na.State,y=na.StationId,name='NA',
                     marker_color='rgb(119, 158, 203)',
                     marker_line_color='black',
                     marker_line_width=1.5, 
                     opacity=0.9
                    ))
inactive = group[group['Status']=='Inactive']
fig.add_trace(go.Bar(x=inactive.State,y=inactive.StationId,name='Inactive',
                     marker_color='rgb(254, 107, 100)',
                     marker_line_color='black',
                     marker_line_width=1.5, 
                     opacity=0.9
                    ))
fig.update_xaxes(showgrid=False)
fig.update_layout(height=300,template='ggplot2',barmode='stack',title='AQI Stations per State',
                  hovermode='x',
                  paper_bgcolor=BGCOLOR,plot_bgcolor='lightgray',margin=dict(l=20,r=20,t=40,b=20))
fig.show()

In [6]:

df_ind = df_sd.copy()
df_ind['Date'] = pd.to_datetime(df_ind['Date'],format='%Y-%m-%d')
df_ind['Period'] = df_ind.apply(lambda x: 'Before' if (x['Date'] < dt.datetime(2020, 3, 23)) else 'After',axis=1)
df_ind = df_ind.query('Date>="2020-01-01"')
df_ind = df_ind.groupby(['Period','StationId'],as_index=False)['AQI','PM2.5','PM10','O3','CO','SO2','NO2'].mean()
df_ind = df_ind.merge(df_st[['StationId','StationName','State','Latitude','Longitude']],how='inner',on='StationId')

def scale(aqiSeries):
    cmax = aqiSeries.max()
    cmin = aqiSeries.min()
    dt = 1e-5
    good = min((50-cmin)/(cmax-cmin)+dt,1.0)
    satisfactory = min((100-cmin)/(cmax-cmin)+dt,1.0)
    moderate = min((200-cmin)/(cmax-cmin)+dt,1.0)
    poor = min((300-cmin)/(cmax-cmin)+dt,1.0)
    very_poor = min((400-cmin)/(cmax-cmin)+dt,1.0)
    severe = min((500-cmin)/(cmax-cmin)+dt,1.0)

    colorcode = [good,satisfactory,moderate,poor,very_poor,severe]
    colorcode = [0.0 if c<0 else c for c in colorcode]
    colors = ['#77DD77','#33AF13','#F6D20E','#F17700','#FE6B64','#F12424']
    scl = []
    prev = 0
    for i in range(len(colorcode)):
        scl.extend([[prev,colors[i]],[colorcode[i],colors[i]]])
        prev=colorcode[i]
        if colorcode[i]==1.0: break
    if scl[-1][0]!=1.0:
        scl[-1][0]=1.0
    
    return scl

<ipython-input-6-3d415357315a>:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



# Web scraping code (do not run)

In [ ]:
from selenium import webdriver
from time import sleep
import re

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
#options.add_argument('user-data-dir=' + r'C:\Users\shiva\Desktop\AQI mp\chromedriver.exe')

driver = webdriver.Chrome(
    executable_path=r'C:\Users\shiva\Desktop\AQI mp\chromedriver.exe',
    options=options)
driver.get('https://www.google.com/search?q=Secretariat, Amaravati - APPCB latitude')
text = driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div[1]/div[1]/div/div[2]/div').text
latitude , longitude = re.findall(r"[-+]?\d*\.?\d+|[-+]?\d+",text)

In [ ]:
def fill_coordinates(row):
    try:
        try:
            driver.get('https://www.google.com/search?q={} coordinates'.format(row.StationName))
            sleep(1)
            text = driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div[1]/div[1]/div/div[2]/div').text
        except:
            driver.get('https://www.google.com/search?q={} coordinates'.format(row.StationName.split('-')[0]))
            sleep(1)
            text = driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div[1]/div[1]/div/div[2]/div').text

        row.Latitude, row.Longitude = re.findall(r"[-+]?\d*\.?\d+|[-+]?\d+",text)
        print(row.Latitude, row.Longitude)
    except:
        pass
    return row

In [ ]:
df_st = df_st.apply(fill_coordinates, axis='columns')
df_st.to_csv('stations_updated')

# plot on map using mapbox

In [9]:
dict_center_zoom={
    'India':[(20.5937,78.9629),2.5],
    #'Bengaluru':[(12.9716,77.5946),9],
    #'Delhi':[(28.7041,77.1025),8],
    #'Mumbai':[(19.0760,72.8777),8],
    #'Hyderabad':[(17.3850,78.4867),8],
    #'Chennai':[(13.0827,80.2707),9]
}

def draw_aqi_map(df,city):
    if city=='India':
        df0 = df
    else:
        if city=='Bengaluru':
            state='Karnataka'
        elif city=='Mumbai':
            state='Maharashtra'
        elif city=='Hyderabad':
            state='Telangana'
        elif city=='Chennai':
            state='Tamil Nadu'
        else:
            state=city
        df0 = df[df['State']==state]
    

    fig = go.Figure()

    df1=df0[df0['Period']=='Before']
    fig.add_trace(go.Scattermapbox(name='Before Lockdown',
        lat=df1.Latitude,
        lon=df1.Longitude,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=17,
            color=df1.AQI,
            colorscale=scale(df1.AQI),
            opacity=0.7
        ),
        text=df1.StationId.astype(str)+'<br><b>Station</b>: '+df1.StationName+'<br><b>AQI</b>: '+np.round(df1.AQI).astype(str),
        hoverinfo='text',
        subplot='mapbox'
    ))

    df2=df0[df0['Period']=='After']
    fig.add_trace(go.Scattermapbox(name='After Lockdown',
        lat=df2.Latitude,
        lon=df2.Longitude,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=17,
            color=df2.AQI,
            colorscale=scale(df2.AQI),
            opacity=0.7
        ),
        text=df2.StationId.astype(str)+'<br><b>Station</b>: '+df2.StationName+'<br><b>AQI</b>: '+np.round(df2.AQI).astype(str),
        hoverinfo='text',
        subplot='mapbox2'
    ))

    fig.update_layout(
        title=city + ': Before & After Lockdown',
        paper_bgcolor=BGCOLOR,
        margin=dict(l=20,r=20,t=40,b=20),
        showlegend=False,
        autosize=True,
        hovermode='closest',
        mapbox=dict(accesstoken=secret_value_1,
                    style='outdoors',
#                     layers= [{
#             "below": 'traces',
#             "sourcetype": "raster",
#             "source": [
#                 "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
#             ]}],
            domain={'x': [0, 0.48], 'y': [0, 1]},
                bearing=0,
                center=dict(
                lat=dict_center_zoom[city][0][0],
                lon=dict_center_zoom[city][0][1]
            ),
        pitch=0,
        zoom=dict_center_zoom[city][1]
        ),
        mapbox2=dict(accesstoken=secret_value_1,
            style='outdoors',
            domain={'x': [0.52, 1.0], 'y': [0, 1]},
            bearing=0,
            center=dict(
                lat=dict_center_zoom[city][0][0],
                lon=dict_center_zoom[city][0][1]
            ),
            pitch=0,
            zoom=dict_center_zoom[city][1],
        ),
    )
    return fig
draw_aqi_map(df_ind,'India')

In [8]:
secret_value_1 = 'pk.eyJ1IjoibWVya2VuZGl6YXIiLCJhIjoiY2t0bndsOHJrMDBzMjJvbzQ0NjlybTc0eSJ9.5X4jIj0tANN07r6kPJdiqQ' 